In [ ]:
from funciones import Functions
Functions().create_navBar()

# Bienvenido al Sistema Analítico Hotelero

Este es el punto de partida para descubrir y gestionar información crítica sobre la operación de tu hotel. 

## ¿Qué puedes hacer aquí?

- **Explorar Estadísticas**: Obtén acceso a datos locales y globales con visualizaciones interactivas.
- **Gestión de Registros**: Crea y actualiza registros para mantener tu información al día.
- **Análisis en Profundidad**: Sumérgete en análisis detallados para tomar decisiones informadas.

Selecciona una opción del menú para comenzar.

---


In [1]:
import requests
import json

url = 'https://postgres-app1-075e5eddc52e.herokuapp.com/client'
respuesta = requests.get(url, verify=False)  # Asumiendo que aún estás en un entorno de desarrollo
print("requests.get(https://postgres-app1-075e5eddc52e.herokuapp.com/client)")
print("Estado de la respuesta:", respuesta.status_code)
print("Contenido de la respuesta:", json.dumps(respuesta.json(), indent=4))  # Imprime solo los primeros 10 caracteres para no saturar la salida

requests.get(https://postgres-app1-075e5eddc52e.herokuapp.com/client)
Estado de la respuesta: 200
Contenido de la respuesta: [
    {
        "age": 38,
        "clid": 1,
        "fname": "Lacey",
        "lname": "Thorneley",
        "memberyear": 15
    },
    {
        "age": 79,
        "clid": 2,
        "fname": "Jana",
        "lname": "Di Francesco",
        "memberyear": 1
    },
    {
        "age": 52,
        "clid": 3,
        "fname": "Lanita",
        "lname": "McIlvoray",
        "memberyear": 30
    },
    {
        "age": 31,
        "clid": 4,
        "fname": "Rosalynd",
        "lname": "Budget",
        "memberyear": 8
    },
    {
        "age": 46,
        "clid": 5,
        "fname": "Jean",
        "lname": "Lenahan",
        "memberyear": 2
    },
    {
        "age": 80,
        "clid": 6,
        "fname": "Katleen",
        "lname": "Huxter",
        "memberyear": 14
    },
    {
        "age": 35,
        "clid": 7,
        "fname": "Robbie",
        "lname

C:\Users\JanielNunez\Desktop\ProgramingProjects\Hotel-JJAD-OG\venv\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'postgres-app1-075e5eddc52e.herokuapp.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
